## NCKU 研究人員資料庫存在性檢查

這個 Jupyter Notebook 用於檢查一份姓名列表中的研究人員是否存在於成功大學的研究成果網站中。

### 流程如下：
1. **準備資料**：在標示為 `TODO` 的儲存格中，填入您想要查詢的姓名列表 `input_list`。
2. **執行爬蟲**：程式會遍歷列表中的每個名字，自動產生查詢網址並抓取網頁內容。
3. **解析與判斷**：透過分析網頁的 HTML，判斷是否存在關鍵的 `<h2>Profiles</h2>` 區塊。
4. **分類結果**：將名字分為「存在於系統中」和「不存在於系統中」兩類。
5. **匯出報告**：將最終結果儲存為一個 CSV 檔案，方便後續使用。

### 步驟 1: 匯入必要的函式庫
首先，我們需要匯入用於網路請求、HTML 解析和數據處理的函式庫。

In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import quote
import time
from tqdm import tqdm

print("函式庫匯入成功！")

函式庫匯入成功！


### 步驟 2: 從 PDF 自動讀取姓名並產生 input_list
此步驟會自動讀取 `../input/` 資料夾中的 PDF 檔案，並從中提取姓名，作為接下來要查詢的列表。

In [17]:
import os
import pdfplumber
import pandas as pd

def get_names_from_pdf(pdf_path):
    names = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                df = pd.DataFrame(table[1:], columns=table[0])
                if '姓名' in df.columns:
                    # 移除 '教授' 並過濾掉空值
                    cleaned_names = df['姓名'].str.replace('教授', '', regex=False).dropna()
                    names.extend(cleaned_names.tolist())
    return names

input_dir = '../input/'
pdf_files = [f for f in os.listdir(input_dir) if f.endswith('.pdf')]

if not pdf_files:
    print(f"在 '{input_dir}' 中找不到任何 PDF 檔案。請確認檔案已放置。")
    input_list = [] # 如果找不到檔案，則使用空列表
else:
    pdf_path = os.path.join(input_dir, pdf_files[0])
    print(f"正在從 {pdf_path} 讀取姓名...")
    input_list = get_names_from_pdf(pdf_path)

print(f"成功讀取 {len(input_list)} 個姓名。")
print("預覽前 5 個姓名:", input_list[:5])

正在從 ../input/名譽教授.pdf 讀取姓名...
成功讀取 126 個姓名。
預覽前 5 個姓名: ['王廷山', '吳添壽', '李克讓', '黃定加', '崔永晉']


### 步驟 3: 定義核心檢查函式
這個函式 `check_profile_exists` 負責處理單一姓名的查詢、爬取和判斷邏輯。

In [18]:
def check_profile_exists(name):
    base_url = "https://researchoutput.ncku.edu.tw/en/searchAll/index/?search={search_term}&pageSize=25&showAdvanced=false&allConcepts=true&inferConcepts=true&searchBy=PartOfNameOrTitle"
    encoded_name = quote(name)
    search_url = base_url.format(search_term=encoded_name)
    
    # 偽裝成一個真實的瀏覽器 User-Agent
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    try:
        response = requests.get(search_url, headers=headers,timeout=15)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        section_titles = soup.find_all("h2", class_="section-title")
        for title in section_titles:
            if "Profiles" in title.get_text(strip=True):
                return True
        return False
    except requests.exceptions.RequestException as e:
        if isinstance (e, requests.exceptions.HTTPError) and e.response.status_code == 403:
            print(f"❌ 請求被拒絕 (403 Forbidden)。網站可能已封鎖爬蟲。")
        else:
            print(f"查詢 '{name}' 時發生網路錯誤: {e}")
        return False

### 步驟 4: 執行查詢與分類
現在我們將遍歷 `input_list`，對每個姓名執行檢查，並將結果存入兩個不同的列表中。

In [12]:
found_list = []
not_found_list = []

for name in tqdm(input_list, desc="正在查詢進度"):
    print(f"正在查詢: {name}...", end="")
    if check_profile_exists(name):
        print("✔ 存在於系統中")
        found_list.append(name)
    else:
        print("❌ 不存在於系統中")
        not_found_list.append(name)
    time.sleep(0.5)

print("--- 查詢完成---")
print(f"存在於系統中的人員 ({len(found_list)}): {found_list}")
print(f"不存在於系統中的人員 ({len(not_found_list)}): {not_found_list}")

### 步驟 5: 匯出結果為獨立的 CSV 檔案
最後，我們使用 `pandas` 將兩個列表合併成一個結構化的 DataFrame，並將其儲存為 CSV 檔案。

In [19]:
found_df = pd.DataFrame({'Name': found_list, 'Status': '存在於系統中'})
not_found_df = pd.DataFrame({'Name': not_found_list, 'Status': '不存在於系統中'})
result_df = pd.concat([found_df, not_found_df], ignore_index=True)

# 建立 "存在" 的 DataFrame
found_df_separate = pd.DataFrame({'姓名': found_list})
found_filename = '../results/存在於系統中的人員.csv' 
found_df_separate.to_csv(found_filename, index=False, encoding='utf-8-sig')
print(f"成功匯出檔案: {found_filename}")

# 建立 "不存在" 的 DataFrame
not_found_df_separate = pd.DataFrame({'姓名': not_found_list})
not_found_filename = '../results/不存在於系統中的人員.csv'
not_found_df_separate.to_csv(not_found_filename, index=False, encoding='utf-8-sig')
print(f"成功匯出檔案: {not_found_filename}")

成功匯出檔案: ../results/存在於系統中的人員.csv
成功匯出檔案: ../results/不存在於系統中的人員.csv


### 步驟 6: 最終結果統計

此處總結了本次執行的所有數據，方便快速了解結果。

In [20]:
total_from_pdf = len(input_list)
total_found = len(found_list)
total_not_found = len(not_found_list)

print("--- 最終統計結果---")
print(f"從 PDF 檔案中總共讀取到 {total_from_pdf} 個姓名。")
print(f"經查詢後，存在於系統中的人員共有 {total_found} 位。")
print(f"經查詢後，不存在於系統中的人員共有 {total_not_found} 位。")

--- 最終統計結果---
從 PDF 檔案中總共讀取到 126 個姓名。
經查詢後，存在於系統中的人員共有 49 位。
經查詢後，不存在於系統中的人員共有 77 位。
